In [1]:

import cv2
#from google.colab.patches import cv2_imshow
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from statistics import stdev,mean


#define globals required through out the whole program
edges=[] #containing all edge tuple
attrs=[] #countaining list of attribute of all nodes
graph_id=1 #id of latest graph
node_id=1 #id of latest node
graph_indicator=[] #containing graph-id for each node
node_labels=[] #containing labels for all node
graph_labels=[]#containing labels for all graph

#activity-label vs activity-name mapping (4-class)
activity_map={}
activity_map[1]='glioma'
activity_map[2]='notumor'
activity_map[3]='meningioma'
activity_map[4]='pituitary'

#z-score normalization
def normalize(arr):
    arr=np.array(arr)
    m=np.mean(arr)
    s=np.std(arr)
    return (arr-m)/s

#generate graph for a given edge-image file
def generate_graphs(filename,node_label,activity_map):
    print(" ... Reading image: "+filename+" ...")
    global node_id,edges,attrs,graph_id,node_labels,graph_indicator
    cnt=0
    img=cv2.imread(filename)
    dim1,dim2,_=img.shape
    attrs1=[]

    print("Image type: " + activity_map[node_label] + "\nPixel matrix is of: "+str(dim1)+"x"+str(dim2))
    img1=img.copy()
    nodes=np.full((dim1,dim2),-1)
    edge=0
    for i in range(dim1):
        for j in range(dim2):
      #considering pixel as node if pixel-value>=128
            b,_,_=img[i][j]
            if(b>=128):
                nodes[i][j]=node_id
                attrs1.append(b)
                graph_indicator.append(graph_id)
                node_labels.append([node_label,activity_map[node_label]])
                node_id+=1
                cnt+=1
            else:
                img1[i][j]=0
    for i in range(dim1):
        for j in range(dim2):
      #forming edge between all adjacent pixels which are node
          if(nodes[i][j]!=-1):
            li=max(0,i-1)
            ri=min(i+2,dim1)
            lj=max(0,j-1)
            rj=min(j+2,dim2)
            for i1 in range(li,ri):
                for j1 in range(lj,rj):
                    if((i1!=i or j1!=j) and (nodes[i1][j1]!=-1)):
                        edges.append([nodes[i][j],nodes[i1][j1]])
                        edge+=1
    attrs1=normalize(attrs1)
    attrs.extend(attrs1)
    del attrs1
    print("For given image nodes formed: "+str(cnt)+" edges formed: "+str(edge))
    if(cnt!=0): graph_id+=1

#generate graphs for all edge-image under given dir along with proper label
def generate_graph_with_labels(dirname,label,activity_map):
    print("\n... Reading Directory: "+dirname+" ...\n")
    global graph_labels
    filenames=glob.glob(dirname+'/*.jpg')
    for filename in filenames:
        generate_graphs(filename,label,activity_map)
        graph_labels.append([label,activity_map[label]])

#generate graphs for all directories
def process_graphs(covid_dir,norm_dir,bac_dir,vir_dir,activity_map):
    global node_labels,graph_labels
    generate_graph_with_labels(covid_dir,1,activity_map)
    generate_graph_with_labels(norm_dir,2,activity_map)
    generate_graph_with_labels(bac_dir,3,activity_map)
    generate_graph_with_labels(vir_dir,4,activity_map)
    print("Processing done")
    print("Total nodes formed: "+str(len(node_labels))+"Total graphs formed: "+str(len(graph_labels)))

#working directories
covid_dir='brain4/Pretwitt/glioma' #for covid
norm_dir='brain4/Pretwitt/notumor' #for normal
bac_dir='brain4/Pretwitt/meningioma' #for bacterial-pneumonia
vir_dir='brain4/Pretwitt/pituitary' #for viral-pneumonia

process_graphs(covid_dir,norm_dir,bac_dir,vir_dir,activity_map)

#check all the lengths of globals
#comment if not necessary
print(len(node_labels))
print(len(graph_labels))
print(len(edges))
print(len(attrs))

#create adjacency dataframe
df_A=pd.DataFrame(columns=["node-1","node-2"],data=np.array(edges))
print("Shape of edge dataframe: "+str(df_A.shape))
print("\n--summary of dataframe--\n" ,df_A.head(50))

#create node label dataframe
df_node_label=pd.DataFrame(data=np.array(node_labels),columns=["label","activity-name"])
print("shape of node-label dataframe: "+str(df_node_label.shape))
print("\n--summary of dataframe--\n" ,df_node_label)

#create graph label dataframe
df_graph_label=pd.DataFrame(data=np.array(graph_labels),columns=["label","activity-name"])
print("shape of node-label dataframe: "+str(df_graph_label.shape))
print("\n--summary of dataframe--\n" ,df_graph_label.head(50))

#create node-attribute dataframe (normalized grayscale value)
df_node_attr=pd.DataFrame(data=np.array(attrs),columns=["gray-val"])
print("shape of node-attribute dataframe: "+str(df_node_attr.shape))
print("\n--summary of dataframe--\n" ,df_node_attr.head(50))

#create graph-indicator datframe
df_graph_indicator=pd.DataFrame(data=np.array(graph_indicator),columns=["graph-id"])
print("shape of graph-indicator dataframe: "+str(df_graph_indicator.shape))
print("\n--summary of dataframe--\n" ,df_graph_indicator.head(50))

#omit activity name later for graph-label and node-label
#since GIN model will only accept the label
df_node_label=df_node_label.drop(["activity-name"],axis=1)
print(df_node_label.head(50))

df_graph_label=df_graph_label.drop(["activity-name"],axis=1)
print(df_graph_label.head(50))



def save_dataframe_to_txt(df,filepath):
    df.to_csv(filepath,header=None,index=None,sep=',',mode='w')



#save all the dataframes to .txt file
#path name: .../GraphTrain/dataset/<dataset_name>/raw/<dataset_name>_<type>.txt
# <type>:
# A--> adjancency matrix
#graph_indicator--> graph-ids of all node
#graph_labels--> labels for all graph
#node_attributes--> attribute(s) for all node
#node_labels--> labels for all node

sourcepath='brain4/Github_Pretwitt/raw'
save_dataframe_to_txt(df_A,sourcepath+'/Github_4class_Pretwitt_A.txt')
save_dataframe_to_txt(df_graph_indicator,sourcepath+'/Github_4class_Pretwitt_graph_indicator.txt')
save_dataframe_to_txt(df_graph_label,sourcepath+'/Github_4class_Pretwitt_graph_labels.txt')
save_dataframe_to_txt(df_node_attr,sourcepath+'/Github_4class_Pretwitt_node_attributes.txt')
save_dataframe_to_txt(df_node_label,sourcepath+'/Github_4class_Pretwitt_node_labels.txt')


... Reading Directory: brain4/Pretwitt/glioma ...

 ... Reading image: brain4/Pretwitt/glioma\img-1.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 9463 edges formed: 45800
 ... Reading image: brain4/Pretwitt/glioma\img-10.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 10499 edges formed: 54146
 ... Reading image: brain4/Pretwitt/glioma\img-100.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 9195 edges formed: 42490
 ... Reading image: brain4/Pretwitt/glioma\img-101.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 6433 edges formed: 31452
 ... Reading image: brain4/Pretwitt/glioma\img-102.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 8546 edges formed: 41102
 ... Reading image: brain4/Pretwitt/glioma\img-103.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 9003 edge

For given image nodes formed: 5375 edges formed: 25846
 ... Reading image: brain4/Pretwitt/glioma\img-145.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 3719 edges formed: 17760
 ... Reading image: brain4/Pretwitt/glioma\img-146.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 8268 edges formed: 40596
 ... Reading image: brain4/Pretwitt/glioma\img-147.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 9942 edges formed: 51042
 ... Reading image: brain4/Pretwitt/glioma\img-148.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 7769 edges formed: 37798
 ... Reading image: brain4/Pretwitt/glioma\img-149.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 8488 edges formed: 42076
 ... Reading image: brain4/Pretwitt/glioma\img-15.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 8314 

For given image nodes formed: 9626 edges formed: 51160
 ... Reading image: brain4/Pretwitt/glioma\img-191.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 10794 edges formed: 57582
 ... Reading image: brain4/Pretwitt/glioma\img-192.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 8108 edges formed: 44616
 ... Reading image: brain4/Pretwitt/glioma\img-193.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 10204 edges formed: 51124
 ... Reading image: brain4/Pretwitt/glioma\img-194.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 12936 edges formed: 64762
 ... Reading image: brain4/Pretwitt/glioma\img-195.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 10068 edges formed: 54902
 ... Reading image: brain4/Pretwitt/glioma\img-196.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 

For given image nodes formed: 7672 edges formed: 44134
 ... Reading image: brain4/Pretwitt/glioma\img-237.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 5468 edges formed: 31228
 ... Reading image: brain4/Pretwitt/glioma\img-238.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 6933 edges formed: 32320
 ... Reading image: brain4/Pretwitt/glioma\img-239.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 9697 edges formed: 50808
 ... Reading image: brain4/Pretwitt/glioma\img-24.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 5449 edges formed: 28712
 ... Reading image: brain4/Pretwitt/glioma\img-240.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 6631 edges formed: 34026
 ... Reading image: brain4/Pretwitt/glioma\img-241.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 9046 

For given image nodes formed: 6524 edges formed: 32854
 ... Reading image: brain4/Pretwitt/glioma\img-283.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 6631 edges formed: 35024
 ... Reading image: brain4/Pretwitt/glioma\img-284.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 6528 edges formed: 36220
 ... Reading image: brain4/Pretwitt/glioma\img-285.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1964 edges formed: 8228
 ... Reading image: brain4/Pretwitt/glioma\img-286.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 10044 edges formed: 53674
 ... Reading image: brain4/Pretwitt/glioma\img-287.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 8066 edges formed: 41434
 ... Reading image: brain4/Pretwitt/glioma\img-288.jpg ...
Image type: glioma
Pixel matrix is of: 512x512
For given image nodes formed: 1041

...

For given image nodes formed: 10192 edges formed: 50612
 ... Reading image: brain4/Pretwitt/pituitary\img-183.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 11217 edges formed: 54316
 ... Reading image: brain4/Pretwitt/pituitary\img-184.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 8008 edges formed: 35550
 ... Reading image: brain4/Pretwitt/pituitary\img-185.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 9602 edges formed: 45242
 ... Reading image: brain4/Pretwitt/pituitary\img-186.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 11061 edges formed: 55938
 ... Reading image: brain4/Pretwitt/pituitary\img-187.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 9989 edges formed: 49500
 ... Reading image: brain4/Pretwitt/pituitary\img-188.jpg ...
Image type: pituitary
Pixel matrix is of: 512

For given image nodes formed: 14620 edges formed: 69226
 ... Reading image: brain4/Pretwitt/pituitary\img-227.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 12637 edges formed: 60552
 ... Reading image: brain4/Pretwitt/pituitary\img-228.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 9395 edges formed: 45248
 ... Reading image: brain4/Pretwitt/pituitary\img-229.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 8468 edges formed: 39542
 ... Reading image: brain4/Pretwitt/pituitary\img-23.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 9707 edges formed: 50168
 ... Reading image: brain4/Pretwitt/pituitary\img-230.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 10194 edges formed: 44138
 ... Reading image: brain4/Pretwitt/pituitary\img-231.jpg ...
Image type: pituitary
Pixel matrix is of: 512x

For given image nodes formed: 9130 edges formed: 47164
 ... Reading image: brain4/Pretwitt/pituitary\img-271.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 11285 edges formed: 58870
 ... Reading image: brain4/Pretwitt/pituitary\img-272.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 13344 edges formed: 71304
 ... Reading image: brain4/Pretwitt/pituitary\img-273.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 10992 edges formed: 58894
 ... Reading image: brain4/Pretwitt/pituitary\img-274.jpg ...
Image type: pituitary
Pixel matrix is of: 256x256
For given image nodes formed: 7623 edges formed: 34302
 ... Reading image: brain4/Pretwitt/pituitary\img-275.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 8886 edges formed: 50482
 ... Reading image: brain4/Pretwitt/pituitary\img-276.jpg ...
Image type: pituitary
Pixel matrix is of: 512

For given image nodes formed: 7912 edges formed: 34856
 ... Reading image: brain4/Pretwitt/pituitary\img-45.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 9460 edges formed: 45188
 ... Reading image: brain4/Pretwitt/pituitary\img-46.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 8805 edges formed: 40260
 ... Reading image: brain4/Pretwitt/pituitary\img-47.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 13534 edges formed: 66542
 ... Reading image: brain4/Pretwitt/pituitary\img-48.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 7798 edges formed: 36212
 ... Reading image: brain4/Pretwitt/pituitary\img-49.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 9840 edges formed: 51584
 ... Reading image: brain4/Pretwitt/pituitary\img-5.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For 

For given image nodes formed: 10712 edges formed: 60032
 ... Reading image: brain4/Pretwitt/pituitary\img-90.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 10223 edges formed: 53254
 ... Reading image: brain4/Pretwitt/pituitary\img-91.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 11536 edges formed: 60056
 ... Reading image: brain4/Pretwitt/pituitary\img-92.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 13978 edges formed: 72200
 ... Reading image: brain4/Pretwitt/pituitary\img-93.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 7393 edges formed: 35626
 ... Reading image: brain4/Pretwitt/pituitary\img-94.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
For given image nodes formed: 9654 edges formed: 50284
 ... Reading image: brain4/Pretwitt/pituitary\img-95.jpg ...
Image type: pituitary
Pixel matrix is of: 512x512
